In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import corner
import importlib
import logging
import os
import keras.backend as K
from functions import load_train_data, load_test_data

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
importlib.reload(logging)
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(gpus[1],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=20000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
    print(e)

2 Physical GPUs, 1 Logical GPU


In [2]:
#3flavor_poisson, nsi_poisson
learn_target = 'nsi_poisson'

x_train, y_train, x_val, y_val = load_train_data(learn_target)

In [3]:
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = tf.keras.Sequential([
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)))])
    return prior_model

def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = tf.keras.Sequential([
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ), tfp.layers.MultivariateNormalTriL(n),])
    return posterior_model

In [4]:
def create_bnn_model(train_size):
    inputs = tf.keras.Input(shape=(len(x_train[0]),), name = 'input')
    features = tf.keras.layers.BatchNormalization()(inputs)
    features = tf.keras.layers.Dense(256, activation='relu')(features)
    features = tf.keras.layers.Dense(128, activation='relu')(features)

    for units in [32]:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)
    features = tf.keras.layers.Dense(16, activation='relu')(features)
    features = tf.keras.layers.Dense(len(y_train[0]), activation='linear')(features)
    outputs = features
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

alpha = np.array([1, 20, 1, 1]) if '3flavor' in learn_target else np.array([1, 20, 1, 1, 1, 1, 1])
alpha = alpha/sum(alpha)

def custom_mse(y_true, y_pred):
    loss = K.square(y_pred - y_true)  
    loss = loss * alpha      
    loss = K.sum(loss, axis=1)        
    return loss

In [5]:
bnn = create_bnn_model(len(x_train))
bnn.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()],
)

if learn_target == '3flavor_poisson': bnn.load_weights('./bnn/' + learn_target + '/weight_1.h5')
else: bnn.load_weights('./bnn/' + learn_target + '/weight_1.h5')

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


In [6]:
data_index = 1
data_all = load_test_data(learn_target, data_index)

In [7]:
sample_amount = 100000

if '3flavor' in learn_target:
    prediction_NO = np.array([bnn.predict(data_all[:1, :])[0] for _ in range(sample_amount)])
    prediction_IO = np.array([bnn.predict(data_all[1:2, :])[0] for _ in range(sample_amount)])
    np.savez('contour/' + learn_target + '_NO/bnn.npz', prediction_NO)
    np.savez('contour/' + learn_target + '_IO/bnn.npz', prediction_IO)
else:
    prediction = np.array([bnn.predict(data_all[:1, :])[0] for _ in range(sample_amount)])
    np.savez('contour/' + learn_target +  '_{}/bnn.npz'.format(data_index), prediction)